In [ ]:
!pip install -qq llama-stack

In [ ]:
LLAMA_STACK_SERVER_HOST = os.getenv("LLAMA_STACK_SERVER_HOST", "llamastack-with-config-service.llama-stack.svc.cluster.local")
LLAMA_STACK_SERVER_PORT = os.getenv("LLAMA_STACK_SERVER_PORT", "8321")

In [ ]:
# Replace host and port
client = LlamaStackClient(base_url=f"http://{LLAMA_STACK_SERVER_HOST}:{LLAMA_STACK_SERVER_PORT}")

agent = Agent(
    client,
    # Check with `llama-stack-client models list`
    model="llama-3-2-3b",
    instructions="You are a helpful assistant",
    # Enable both RAG and tool usage
    tools=[
        "builtin::websearch"
    ],
    # Control the inference loop
    max_infer_iters=5,
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 0.7, "top_p": 0.95},
        "max_tokens": 2048,
    },
)
session_id = agent.create_session("monitored_session")




In [ ]:
# Ask about some web search topic
response = agent.create_turn(
    messages=[{"role": "user", "content": "Whats the current openshift release?"}],
    session_id=session_id,
)

# Monitor each step of execution
for log in AgentEventLogger().log(response):
    log.print()


In [ ]:
# Ask about cluster related topic
response = agent.create_turn(
    messages=[{"role": "user", "content": "What namespaces are existing inside the cluster?"}],
    session_id=session_id,
)

# Monitor each step of execution
for log in AgentEventLogger().log(response):
    log.print()